In [2]:
import tensorflow as tf
import numpy as np
from default_settings import *

In [3]:
def placeholder_inputs():
    images_placeholder = tf.placeholder(tf.float32, shape=[25,16,112,112,3], name="input_clip")
    labels_placeholder = tf.placeholder(tf.int64, shape=[25], name="labels")
    return images_placeholder, labels_placeholder

def _variable_on_cpu(name, shape, initializer):
    with tf.device('/cpu:0'):
        var = tf.get_variable(name, shape, initializer=initializer)
    return var
def _get_var(name, initializer):
    with tf.device('/cpu:0'):
        var = tf.get_variable(name, initializer=initializer)
    return var

def _variable_with_weight_decay(name, shape, wd):
    var = _variable_on_cpu(name, shape, tf.constant_initializer(0.0))
    if wd is not None:
        weight_decay = tf.multiply(tf.nn.l2_loss(var), wd, name='weight_decay_loss')
        tf.add_to_collection('losses', weight_decay)
    return var

In [4]:
#Loading variable functions
def conv_layer(tensor_in, w_name, b_name, shape_weight, shape_bias, wd, layer_name, weight, bias):
    with tf.variable_scope(layer_name):
        w = _get_var(w_name, initializer=weight)
        b = _get_var(b_name,initializer=bias)
        tf.add_to_collection(tf.GraphKeys.MODEL_VARIABLES, w)
        tf.add_to_collection(tf.GraphKeys.MODEL_VARIABLES, b)
        
        conv = tf.nn.conv3d(tensor_in, w, strides=[1,1,1,1,1], padding='SAME',name='conv')
        conv_b = tf.nn.bias_add(conv, b, name='conv_bias')
        act = tf.nn.relu(conv_b, 'relu')
        tf.summary.histogram('Convolution_Weights', w)
        tf.summary.histogram('Conv_Biases', b)
        tf.summary.histogram('Conv_Activations', act)
        return act

def fc_layer(tensor_in, w_name, b_name, shape_weight, shape_bias, wd, _dropout, layer_name, weight,bias):
    with tf.variable_scope(layer_name):
        w = _get_var(w_name, initializer=weight)
        b = _get_var(b_name,initializer=bias)
        tf.add_to_collection(tf.GraphKeys.MODEL_VARIABLES, w)
        tf.add_to_collection(tf.GraphKeys.MODEL_VARIABLES, b)
        
        fc_1 = tf.matmul(tensor_in, w, name='fc_linear')
        fc_b = tf.nn.bias_add(fc_1, b, name='fc_bias')
        act = tf.nn.relu(fc_b, 'relu')
        tensor_out = tf.nn.dropout(act, _dropout, name='dropout')
        tf.summary.histogram('FC_Weights', w)
        tf.summary.histogram('FC_Biases', b)
        tf.summary.histogram('FC_Activations', act)
        return tensor_out
        
def max_pool_3d(tensor_in, k, layer_name='max_pool'):
    with tf.variable_scope(layer_name):
        k_size=[1,k,2,2,1]
        tensor_out = tf.nn.max_pool3d(tensor_in, ksize=k_size, strides=k_size, padding='SAME')
        return tensor_out
    
def c3d_model_pre(_X, _dropout, batch_size, wd, weights, biases):
    
    #Convolution 1a
    conv1 = conv_layer(_X, 'wc1a', 'bc1a', [3, 3, 3, 3, 64], [64], wd, 'Convolution_1a', weights['wc1'], biases['bc1'])
    pool1 = max_pool_3d(conv1, 1, 'Max_Pooling_1')
    
    #Convolution 2a
    conv2 = conv_layer(pool1, 'wc2a', 'bc2a', [3, 3, 3, 64, 128], [128], wd, 'Convolution_2a', weights['wc2'], biases['bc2'])
    pool2 = max_pool_3d(conv2, 2, 'Max_Pooling_2')
    
    #Convolution 3a, 3b
    conv3a = conv_layer(pool2, 'wc3a', 'bc3a',  [3, 3, 3, 128, 256], [256], wd, 'Convolution_3a', weights['wc3a'], biases['bc3a'])
    conv3b = conv_layer(conv3a, 'wc3b', 'bc3b', [3, 3, 3, 256, 256], [256], wd, 'Convolution_3b', weights['wc3b'], biases['bc3b'])
    pool3 = max_pool_3d(conv3b, 2, 'Max_Pooling_3')
    
    #Convolution 4a, 4b
    conv4a = conv_layer(pool3, 'wc4a', 'bc4a',  [3, 3, 3, 256, 512], [512], wd, 'Convolution_4a', weights['wc4a'], biases['bc4a'])
    conv4b = conv_layer(conv4a, 'wc4b', 'bc4b', [3, 3, 3, 512, 512], [512], wd, 'Convolution_4b', weights['wc4b'], biases['bc4b'])
    pool4 = max_pool_3d(conv4b, 2, 'Max_Pooling_4')
    
    #Convolution 4a, 4b
    conv5a = conv_layer(pool4, 'wc5a', 'bc5a',  [3, 3, 3, 512, 512], [512], wd, 'Convolution_5a', weights['wc5a'], biases['bc5a'])
    conv5b = conv_layer(conv5a, 'wc5b', 'bc5b', [3, 3, 3, 512, 512], [512], wd, 'Convolution_5b', weights['wc5b'], biases['bc5b'])
    pool5 = max_pool_3d(conv5b, 2, 'Max_Pooling_5')

    #Reshape weights for fc6
    pool5 = tf.transpose(pool5, perm=[0,1,4,2,3])
    dense1 = tf.reshape(pool5, [batch_size, 8192])

    #FC6 and FC7
    dense1 = fc_layer(dense1, 'wd1', 'bd1', [8192, 4096], [4096], wd, _dropout, 'FC6', weights['wd1'], biases['bd1'])
    dense2 = fc_layer(dense1, 'wd2', 'bd2', [4096, 4096], [4096], wd, _dropout, 'FC7', weights['wd2'], biases['bd2'])
    
    with tf.variable_scope('Softmax_Linear'):
        w_out = _get_var('wout', initializer=weights['out'])
        b_out = _get_var('bout',initializer=biases['out'])
        tf.add_to_collection(tf.GraphKeys.MODEL_VARIABLES, w_out)
        tf.add_to_collection(tf.GraphKeys.MODEL_VARIABLES, b_out)
        out = tf.matmul(dense2, w_out) + b_out
    
    return out

In [5]:
def create_graph(model_settings, weights, biases):
    images_placeholder, labels_placeholder = placeholder_inputs()
    model_out = c3d_model_pre(images_placeholder, 0.5, 
                                         25,
                                         model_settings['weight_decay'],
                                         weights,
                                         biases)  

In [12]:
weight_decay = 0.0005
num_class = 101
with tf.Graph().as_default(), tf.device('/cpu:0'):
    with tf.variable_scope('var_name') as var_scope:
        weights = {
              'wc1': _variable_with_weight_decay('wc1', [3, 3, 3, 3, 64], weight_decay),
              'wc2': _variable_with_weight_decay('wc2', [3, 3, 3, 64, 128], weight_decay),
              'wc3a': _variable_with_weight_decay('wc3a', [3, 3, 3, 128, 256], weight_decay),
              'wc3b': _variable_with_weight_decay('wc3b', [3, 3, 3, 256, 256], weight_decay),
              'wc4a': _variable_with_weight_decay('wc4a', [3, 3, 3, 256, 512], weight_decay),
              'wc4b': _variable_with_weight_decay('wc4b', [3, 3, 3, 512, 512], weight_decay),
              'wc5a': _variable_with_weight_decay('wc5a', [3, 3, 3, 512, 512], weight_decay),
              'wc5b': _variable_with_weight_decay('wc5b', [3, 3, 3, 512, 512], weight_decay),
              'wd1': _variable_with_weight_decay('wd1', [8192, 4096], weight_decay),
              'wd2': _variable_with_weight_decay('wd2', [4096, 4096], weight_decay),
              'out': _variable_with_weight_decay('wout', [4096, num_class], weight_decay)
        }
        biases = {
              'bc1': _variable_with_weight_decay('bc1', [64], 0.000),
              'bc2': _variable_with_weight_decay('bc2', [128], 0.000),
              'bc3a': _variable_with_weight_decay('bc3a', [256], 0.000),
              'bc3b': _variable_with_weight_decay('bc3b', [256], 0.000),
              'bc4a': _variable_with_weight_decay('bc4a', [512], 0.000),
              'bc4b': _variable_with_weight_decay('bc4b', [512], 0.000),
              'bc5a': _variable_with_weight_decay('bc5a', [512], 0.000),
              'bc5b': _variable_with_weight_decay('bc5b', [512], 0.000),
              'bd1': _variable_with_weight_decay('bd1', [4096], 0.000),
              'bd2': _variable_with_weight_decay('bd2', [4096], 0.000),
              'out': _variable_with_weight_decay('bout', [num_class], 0.000),
        }
    saver_old_vals = tf.train.Saver(list({**weights, **biases}.values()))
    
    create_graph(model_settings, weights, biases)
    
    init_all = tf.variables_initializer(list({**weights, **biases}.values()))
    init_model = tf.variables_initializer(tf.model_variables())
    
    saver_new_vars = tf.train.Saver(tf.model_variables())
    
    model_vars = tf.model_variables()
    with tf.Session(config=tf.ConfigProto(allow_soft_placement=True)) as sess:
        sess.run(init_all)
        saver_old_vals.restore(sess, './models/c3d_ucf101_finetune_whole_iter_20000_TF.model')
        sess.run(init_model)
        variables = sess.run(model_vars)
        wc1a = variables[0]
        wc1 = sess.run(weights['wc1'])
        print(wc1a.shape)
        print(wc1a[1,:,:,1,1])
        print(wc1[1,:,:,1,1])
        print('bias last', sess.run(model_vars[-1]))
        #saver_new_vars.save(sess, './models/s1m-ucf101.model', write_meta_graph=False)
        print('Saved!')

INFO:tensorflow:Restoring parameters from ./models/c3d_ucf101_finetune_whole_iter_20000_TF.model
(3, 3, 3, 3, 64)
[[-0.3156917   0.32323226  0.26934817]
 [-0.65775526  0.11280221  0.5430222 ]
 [-0.4000663  -0.24700063  0.30445343]]
[[-0.3156917   0.32323226  0.26934817]
 [-0.65775526  0.11280221  0.5430222 ]
 [-0.4000663  -0.24700063  0.30445343]]
bias last [ 8.74494377e-04 -1.35114812e-03  6.59827027e-04  6.54941017e-04
  7.31361433e-05 -3.76383949e-04 -6.78170531e-04  5.84213063e-04
 -8.07066972e-04 -4.97078814e-04 -6.45212247e-04 -9.56462347e-04
  2.01541370e-05 -2.82382651e-04  3.79555597e-04 -5.21573762e-04
  8.45658797e-05  1.93414729e-04 -7.21632969e-04  6.79509103e-05
 -1.28937862e-03  1.16138393e-03 -2.79989879e-04 -6.56171644e-04
  5.64430782e-04 -3.56820325e-04 -6.81955658e-04  1.15257782e-04
  3.14484729e-04  3.77514982e-04 -2.23722134e-04  4.23660140e-06
  7.54467183e-05  7.73004023e-04  1.35898113e-03  5.93795208e-04
 -6.05327194e-04  3.56549892e-04  7.54304638e-04  6.755